In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import re

#### Using HeFESTo lookup table in aspect

In this note book, I'll show how to:

* a. Import a HeFESTo lookup table
* b. Plot This lookup table
* c. Estimate the amount of density change and latent heat generation on a mantle adiabat
* d. convert to a "perplex" lookup table readable by aspect
* e. Maintain a workflow using the lookup table, including how to check your implementation.

Note on the example lookup table I use:
(todo)

#### Step 1: Define a class to read in a lookup table

Here I will first define a class to read the header and the contents of a lookup table.
Upon calling the member function, this class saves the information of the header and the data into class objects.
Later on, I will include more features in the class to perform other operations, thus I only call this class "HEFESTO_0" here.

The header information is saved in a dictionary while is printed at the end of the cell below.
The data is saved into a 2d numpy array.

In [ ]:
def ReadHeader(fin):
    '''
    Read header information from file.
    An example of string is:
        'P	   Depth  T(K)'
    Inputs:
        fin: a file handler (read)
    Returns:
        _header(dict): header information
            key(dict): infomation under this key:
                'col':
                    column in file
                'unit':
                    unit of this variable
    '''
    header = {}
    first_line = fin.readline()
    entries = first_line.split(' ')
    i = 0
    for entry in entries:
        entry = entry.strip(' \t\n')
        if len(entry) > 0:
            col = i
            temp = entry.split('(')
            _name = temp[0].strip(' \t')
            if len(temp) == 1:
                unit = None
            elif len(temp) == 2:
                temp1 = temp[1].lstrip(' \t')
                unit = temp1.strip(' \t)')
            else:
                raise ValueError("Entry %s is not understood" % entry)
            header[_name] = {'unit': unit, 'col': i}
            i += 1
    header['total'] = i # total number of columns in file
    return header


class HEFESTO_0():

    def __init__(self):
        '''
        initiate class
        '''
        self.header = {}
        self.data = []
        self.version = "1.0.0"
        self.min1 = 0.0 
        self.delta1 = 0.0 
        self.number1 = 0
        self.min2 = 0.0
        self.delta2 = 0.0
        self.number2 = 0
        self.indexes = []  # indexes of output data
        self.number_out1 = 0 # number of output
        self.number_out2 = 0
        self.delta_out1 = 0.0  # intervals used to outptu
        self.delta_out2 = 0.0 
        self.oheader = { 'Temperature': 'T(K)',  'Pressure': 'P(bar)' ,  'Density': 'rho,kg/m3',\
        'Thermal_expansivity': 'alpha,1/K', 'Isobaric_heat_capacity': 'cp,J/K/kg',\
        'VP': 'vp,km/s', 'VS': 'vs,km/s', 'Enthalpy': 'h,J/kg' }
        # unit to output
        self.ounit = {'Temperature': 'K', 'Pressure': 'bar', 'Thermal_expansivity': '1/K',\
        'Isobaric_heat_capacity': 'J/K/kg', 'Density': 'kg/m3', 'VP':'km/s', 'VS':'km/s', 'Enthalpy': 'J/kg'}

    def read_table(self, path):
        '''
        read data
        '''
        print("Reading Header: %s" % path)
        with open(path, 'r') as fin:
            self.header = ReadHeader(fin)

        print("Reading Data: %s" % path)
        self.data = np.genfromtxt(path, comments='#', filling_values=0.0)


hfile = "/home/lochy/ASPECT_PROJECT/aspectLib/large_data_files/Lookup2011/pyrolite.56"    
assert(os.path.isfile(hfile))
# call processfunction
Hefesto = HEFESTO_0()
Hefesto.read_table(hfile)
print("File header: ")
print(Hefesto.header)  # debug
print("Size of file data")
print(Hefesto.data.shape)